# colab용 자동화

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import pickle
# from soynlp.hangle import levenshtein
# # from PreProcessing.find_common_part
# from konlpy.tag import *
# from PreProcessing import find_common_part
import numpy as np
from gensim.models import Word2Vec
from keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# 불러올때    
with open('/content/drive/Shared drives/BigData/team__KU/data/result.txt', 'rb') as f:
    result = pickle.load(f)

result_gensim_input = [_.split() for _ in result if _ != '']
result_tokenizer_input = [v for i, v in enumerate(result) if i%2 == 0 and v != '']
ebs_in_result_for_getting_max_len = [v for i, v in enumerate(result_gensim_input) if i%2 == 0 and v != []]
ebs_in_result_gensim_input = [v for i, v in enumerate(result_gensim_input) if i%2 == 0 and v != [] ]
google_in_result_gensim_input = [v for i, v in enumerate(result_gensim_input) if i%2 == 1 and v != []]
    
# Generate EBS string vectors matrix
ws = 1
es = 30

model_cbow = Word2Vec(
                ebs_in_result_gensim_input, 
                window = ws,
                size =es,
                min_count=1,
                workers = 10
                )
vocabs = list(model_cbow.wv.index2word)
embedding_matrix = np.zeros((len(vocabs), es))
for i, w in enumerate(vocabs):
    embedding_matrix[i] = model_cbow[w]

#embedding_matrix 맨 위에 0벡터 추가
stacked_zero = np.zeros((1, es))
embedding_matrix = np.vstack((stacked_zero, embedding_matrix))

t = Tokenizer()
t.fit_on_texts(result_tokenizer_input)
vocab_size = len(t.word_index) + 1

#시퀀스 만들기
sequences = list()

for line in result_tokenizer_input: # 1,214 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[i-9 if i > 9 else 0:i+1]
        sequences.append(sequence)
        
max_len=max(len(l) for l in sequences)
print('max length : {}'.format(max_len))

sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

y = to_categorical(y, num_classes=vocab_size)

embedding_layer = Embedding(vocab_size,
                            es,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=False)

model = Sequential()
model.add(embedding_layer)
# y데이터를 분리하였으므로 이제 X데이터의 길이는 기존 데이터의 길이 - 1
model.add(LSTM(128, activation = 'relu'))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=50, verbose=2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


max length : 10
Epoch 1/50
1558/1558 - 40s - loss: 7.7710 - accuracy: 0.0160
Epoch 2/50
1558/1558 - 40s - loss: 7.4106 - accuracy: 0.0204
Epoch 3/50
1558/1558 - 40s - loss: 7.2496 - accuracy: 0.0242
Epoch 4/50
1558/1558 - 40s - loss: 7.1129 - accuracy: 0.0248
Epoch 5/50
1558/1558 - 40s - loss: 6.9853 - accuracy: 0.0260
Epoch 6/50
1558/1558 - 40s - loss: 6.8546 - accuracy: 0.0283
Epoch 7/50
1558/1558 - 40s - loss: 6.7244 - accuracy: 0.0292
Epoch 8/50
1558/1558 - 40s - loss: 6.5928 - accuracy: 0.0299
Epoch 9/50
1558/1558 - 40s - loss: 6.4477 - accuracy: 0.0303
Epoch 10/50
1558/1558 - 40s - loss: 6.2696 - accuracy: 0.0327
Epoch 11/50
1558/1558 - 40s - loss: 6.0690 - accuracy: 0.0343
Epoch 12/50
1558/1558 - 40s - loss: 5.8477 - accuracy: 0.0372
Epoch 13/50
1558/1558 - 40s - loss: 5.6047 - accuracy: 0.0428
Epoch 14/50
1558/1558 - 40s - loss: 5.3621 - accuracy: 0.0641
Epoch 15/50
1558/1558 - 40s - loss: 5.1390 - accuracy: 0.1000
Epoch 16/50
1558/1558 - 40s - loss: 4.9465 - accuracy: 0.1346
E

# --------------------------------------------------------------------

In [ ]:
# for _ in list(range(10,160,10)):
model_skip = Word2Vec(
result_gensim_input, 
window = 2,
size =100,
min_count=1,
workers = 10
)
print(_)
print('집값삼각형 : ', model_skip.most_similar('집값삼각형',topn=5))
print()
print('도영 : ', model_skip.most_similar('도영',topn=5))
print()
print('싸인 : ', model_skip.most_similar('싸인',topn=5))
print()
print('수박 : ',model_skip.most_similar('수박',topn=5)) 
print()
print('가게대변인 : ',model_skip.most_similar('가게대변인',topn=5))
print()

In [ ]:
# model_skip = Word2Vec(
#     result_gensim_input, 
#     window = 2,
#     size =40,
#     min_count=1,
#     sg = 1,
#     workers = 10
# )
# print(model_skip)

In [ ]:
model_skip.most_similar('집값삼각형',topn=10)

In [ ]:
model_skip.most_similar('도영',topn=10)

In [ ]:
model_skip.most_similar('싸인',topn=10)

In [ ]:
model_skip.most_similar('수박',topn=10) #특수각

In [ ]:
model_skip.most_similar('가게대변인',topn=10) #각의대변

In [ ]:
model_skip.most_similar('추위',topn=10) #최애

In [ ]:
model_skip.most_similar('',topn=10)

In [ ]:
model_skip.most_similar('',topn=10)

In [ ]:
model_skip.most_similar('',topn=10)

In [ ]:
model_skip.most_similar('',topn=10)

In [ ]:
model_skip.most_similar('',topn=10)